# Midcurve by Llama 2

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig
!pip install ludwig[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.3 MB/s eta 0:0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 21.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a1b2e0e970e239bc334def6fd3aa2c1aff1a15cb8d3b43ce2fcd8a5fb27fb8ed
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [3]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    model = None
    torch.cuda.empty_cache()

In [5]:
import getpass
import locale; locale.getpreferredencoding = lambda: "UTF-8"
import logging
import os
import torch
import yaml

from ludwig.api import LudwigModel


os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [7]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks')

Mounted at /content/drive


In [8]:
from google.colab import data_table; data_table.enable_dataframe_formatter()
import numpy as np; np.random.seed(123)
import pandas as pd

In [9]:
df = pd.read_csv('/content/drive/MyDrive/ImpDocs/Work/AICoach/Notebooks/data/shapes2brep.csv')
df.head()

,ShapeName,Profile,Midcurve,Profile_brep,Midcurve_brep
0,I,"[[5.0, 5.0], [10.0, 5.0], [10.0, 20.0], [5.0, ...","[[7.5, 5.0], [7.5, 20.0]]","{""Points"": [[5.0, 5.0], [10.0, 5.0], [10.0, 20...","{""Points"": [[7.5, 5.0], [7.5, 20.0]], ""Lines"":..."
1,L,"[[5.0, 5.0], [10.0, 5.0], [10.0, 30.0], [35.0,...","[[7.5, 5.0], [7.5, 32.5], [35.0, 32.5]]","{""Points"": [[5.0, 5.0], [10.0, 5.0], [10.0, 30...","{""Points"": [[7.5, 5.0], [7.5, 32.5], [35.0, 32..."
2,Plus,"[[0.0, 25.0], [10.0, 25.0], [10.0, 45.0], [15....","[[12.5, 0.0], [12.5, 22.5], [12.5, 45.0], [0.0...","{""Points"": [[0.0, 25.0], [10.0, 25.0], [10.0, ...","{""Points"": [[12.5, 0.0], [12.5, 22.5], [12.5, ..."
3,T,"[[0.0, 25.0], [25.0, 25.0], [25.0, 20.0], [15....","[[12.5, 0.0], [12.5, 22.5], [25.0, 22.5], [0.0...","{""Points"": [[0.0, 25.0], [25.0, 25.0], [25.0, ...","{""Points"": [[12.5, 0.0], [12.5, 22.5], [25.0, ..."
4,I_scaled_2,"[[10.0, 10.0], [20.0, 10.0], [20.0, 40.0], [10...","[[15.0, 10.0], [15.0, 40.0]]","{""Points"": [[10.0, 10.0], [20.0, 10.0], [20.0,...","{""Points"": [[15.0, 10.0], [15.0, 40.0]], ""Line..."


In [33]:
instruction_tuning_llama2_yaml = yaml.safe_load("""
model_type: llm
base_model: meta-llama/Llama-2-7b-hf

quantization:
 bits: 4

adapter:
 type: lora

prompt:
  template: |
    ### Instruction:
    You are a geometric modeling expert. You need to read 2D profile structure
    called 'Profile_brep' from json format and convert it to corresponding
    2D midcruve strucure called 'Midcurve_brep' also in json format.
    Below is an example:

    ### Input:
    {Profile_brep}

    ### Response:

input_features:
 - name: Profile_brep
   type: text

output_features:
 - name: Midcurve_brep
   type: text

trainer:
 type: finetune
 learning_rate: 0.0003
 batch_size: 1
 gradient_accumulation_steps: 8
 epochs: 3

backend:
 type: local
""")

In [34]:
machine_translation_yaml = yaml.safe_load("""
input_features:
    -   name: Profile_brep
        type: text

output_features:
    -   name: Midcurve_brep
        type: text
        loss:
            type: sequence_softmax_cross_entropy
trainer:
    batch_size: 96
""")

In [35]:
model_instruction_tuning = LudwigModel(config=instruction_tuning_llama2_yaml) # instruction_tuning_llama2_yaml, machine_translation_yaml
results_instruction_tuning = model_instruction_tuning.train(dataset=df)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


OutOfMemoryError: ignored

In [31]:
test_df = pd.DataFrame([
    {
        "Profile_brep": '{"Points": [[12.48, 0.65], [11.31, 23.12], [10.13, 45.59], [-1.18, 22.47], [23.79, 23.78]], "Lines": [[0, 1], [4, 1], [2, 1], [3, 1]], "Segments": [[0], [1], [2], [3]]}'
    },
    {
        "Profile_brep": '{"Points": [[12.48, 0.65], [11.31, 23.12], [23.79, 23.78], [-1.18, 22.47]], "Lines": [[0, 1], [1, 2], [3, 1]], "Segments": [[0], [1], [2]]}'
    }
])

In [32]:
predictions_instruction_tuning_df, output_directory = model_instruction_tuning.predict(dataset=test_df)
shapes_brep_dict_list_strs = predictions_instruction_tuning_df["Midcurve_brep_response"].tolist()
print(shapes_brep_dict_list_strs)

['<SOS> { " points " : [ [ - 7 . 87 , - 7 . 87 , - 29 . 31 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ]', '<SOS> { " points " : [ [ - 7 . 87 , - 7 . 87 , - 29 . 31 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ] , [ - 22 . 02 ] , [ - 10 . 33 81 ]']


In [20]:
import json
shapes_brep_dict_list = json.loads(shapes_brep_dict_list_strs)
print(shapes_brep_dict_list)

TypeError: ignored

In [ ]:
def plot_breps(shapes_brep_dict_list):
    for dct in shapes_brep_dict_list:
        profile_point_list = dct['Profile']
        profile_x_coords, profile_y_coords = zip(*profile_point_list)
        profile_brep = dct['Profile_brep']
        profile_segments = profile_brep["Segments"]
        profile_lines = profile_brep["Lines"]
        profile_segment_color = 'black'
        # Plot Profile segments
        for segment in profile_segments:
            for line_idx in segment:
                line = profile_lines[line_idx]
                x_segment = [profile_x_coords[i] for i in line]
                y_segment = [profile_y_coords[i] for i in line]
                plt.plot(x_segment + [x_segment[0]], y_segment + [y_segment[0]], color=profile_segment_color,
                         marker='o')

        midcurve_point_list = dct['Midcurve']
        midcurve_x_coords, midcurve_y_coords = zip(*midcurve_point_list)
        midcurve_brep = dct['Midcurve_brep']
        midcurve_segments = midcurve_brep["Segments"]
        midcurve_lines = midcurve_brep["Lines"]
        midcurve_segment_color = 'red'

        # Plot Midcurve segments
        for segment in midcurve_segments:
            for line_idx in segment:
                line = midcurve_lines[line_idx]
                x_segment = [midcurve_x_coords[i] for i in line]
                y_segment = [midcurve_y_coords[i] for i in line]
                plt.plot(x_segment + [x_segment[0]], y_segment + [y_segment[0]], color=midcurve_segment_color,
                         marker='x')

        plt.axis('equal')

In [ ]:
plot_breps(shapes_brep_dict_list)